<a href="https://colab.research.google.com/github/JoaoMunhoz01/Algoritmos-de-Ordenacao/blob/master/ChatBotIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


O objetivo desta aula é construirmos um garçom virtual com as seguintes intenções:

    fazer pedido
    ver cardápio
    fechar a conta


# Preparação dos dados
## coleta de dados


In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open('Dados').sheet1


In [4]:

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
df=pd.DataFrame(rows[1:],columns=rows[0])

[['fazer_pedido', 'fechar_conta', 'conjunto', 'ver_cardapio'], ['quero um soba', 'quanto que deu?', 'Treino', 'me veja o cardapio'], ['vou querer 3 sobas', 'quanto vai ser o prejuizo?', 'Teste', 'pode mandar o menu?'], ['quero dois soba', 'quanto que foi?', 'Treino', 'me veja o menu'], ['vou querer 4 sobas', 'manda a conta', 'Teste', 'me veja o cardapio por favor'], ['quero um soba', 'quanto que deu?', 'Treino', 'me veja o cardapio'], ['vou querer 3 sobas', 'quanto vai ser o prejuizo?', 'Teste', 'pode mandar o menu?'], ['quero dois soba', 'quanto que foi?', 'Treino', 'me veja o menu'], ['vou querer 4 sobas', 'manda a conta', 'Teste', 'me veja o cardapio por favor'], ['quero um soba', 'quanto que deu?', 'Treino', 'me veja o cardapio'], ['vou querer 3 sobas', 'quanto vai ser o prejuizo?', 'Teste', 'pode mandar o menu?'], ['quero dois soba', 'quanto que foi?', 'Treino', 'me veja o menu'], ['vou querer 4 sobas', 'manda a conta', 'Teste', 'me veja o cardapio por favor'], ['quero um soba', '

In [5]:
intencoes = ['fazer_pedido', 'fechar_conta', 'ver_cardapio']
xtrain_global = []
ytrain_global = []
for intencao in intencoes:
    lintencao = df[df['conjunto']=='Treino'][intencao].values.tolist()
    xtrain_global += lintencao
    ytrain_global += [intencao]*len(lintencao)


In [6]:
xtrain_global,ytrain_global

(['quero um soba',
  'quero dois soba',
  'quero um soba',
  'quero dois soba',
  'quero um soba',
  'quero dois soba',
  'quero um soba',
  'quero dois soba',
  'quero um soba',
  'quero dois soba',
  'quero um soba',
  'quero dois soba',
  'quero um soba',
  'quero dois soba',
  'quero um soba',
  'quero dois soba',
  'quero um soba',
  'quero dois soba',
  'quero um soba',
  'quero dois soba',
  'quero um soba',
  'quero dois soba',
  'quero um soba',
  'quero dois soba',
  'quero um soba',
  'quero dois soba',
  'quanto que deu?',
  'quanto que foi?',
  'quanto que deu?',
  'quanto que foi?',
  'quanto que deu?',
  'quanto que foi?',
  'quanto que deu?',
  'quanto que foi?',
  'quanto que deu?',
  'quanto que foi?',
  'quanto que deu?',
  'quanto que foi?',
  'quanto que deu?',
  'quanto que foi?',
  'quanto que deu?',
  'quanto que foi?',
  'quanto que deu?',
  'quanto que foi?',
  'quanto que deu?',
  'quanto que foi?',
  'quanto que deu?',
  'quanto que foi?',
  'quanto que deu?

In [7]:
xtest = []
ytest = []
for intencao in intencoes:
    lintencao = df[df['conjunto']=='Teste'][intencao].values.tolist()
    xtest += lintencao
    ytest += [intencao]*len(lintencao)

In [8]:
len(xtest),len(xtrain_global)

(78, 78)

In [9]:
xtest,ytest

(['vou querer 3 sobas',
  'vou querer 4 sobas',
  'vou querer 3 sobas',
  'vou querer 4 sobas',
  'vou querer 3 sobas',
  'vou querer 4 sobas',
  'vou querer 3 sobas',
  'vou querer 4 sobas',
  'vou querer 3 sobas',
  'vou querer 4 sobas',
  'vou querer 3 sobas',
  'vou querer 4 sobas',
  'vou querer 3 sobas',
  'vou querer 4 sobas',
  'vou querer 3 sobas',
  'vou querer 4 sobas',
  'vou querer 3 sobas',
  'vou querer 4 sobas',
  'vou querer 3 sobas',
  'vou querer 4 sobas',
  'vou querer 3 sobas',
  'vou querer 4 sobas',
  'vou querer 3 sobas',
  'vou querer 4 sobas',
  'vou querer 3 sobas',
  'vou querer 4 sobas',
  'quanto vai ser o prejuizo?',
  'manda a conta',
  'quanto vai ser o prejuizo?',
  'manda a conta',
  'quanto vai ser o prejuizo?',
  'manda a conta',
  'quanto vai ser o prejuizo?',
  'manda a conta',
  'quanto vai ser o prejuizo?',
  'manda a conta',
  'quanto vai ser o prejuizo?',
  'manda a conta',
  'quanto vai ser o prejuizo?',
  'manda a conta',
  'quanto vai ser o

## split de treino em validação e treino

In [10]:
ytrain_global[50],xtrain_global[50]

('fechar_conta', 'quanto que deu?')

In [11]:
import sklearn.model_selection as model_selection

In [12]:
xtrain,xval,ytrain,yval = model_selection.train_test_split(xtrain_global,ytrain_global,test_size=0.25,stratify=ytrain_global)

In [13]:
np.unique(ytrain,return_counts=True)

(array(['fazer_pedido', 'fechar_conta', 'ver_cardapio'], dtype='<U12'),
 array([19, 19, 20]))

## pré-processamento

O pré-processamento consiste em transformar todas as sentenças em vetores utilizando sentence-embeddings 

In [14]:

!pip install -U sentence-transformers

     |████████████████████████████████| 79 kB 3.6 MB/s 
     |████████████████████████████████| 3.8 MB 39.4 MB/s 
     |████████████████████████████████| 1.2 MB 43.7 MB/s 
     |████████████████████████████████| 67 kB 4.6 MB/s 
     |████████████████████████████████| 895 kB 35.5 MB/s 
     |████████████████████████████████| 6.5 MB 35.5 MB/s 
     |████████████████████████████████| 596 kB 14.7 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=02fa30aee127cf4ad514a0d3dff82e9e4f6f19ca839fa70ec745d4c4dfd96b08
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [15]:
from sentence_transformers import SentenceTransformer

In [16]:
converter = SentenceTransformer('multi-qa-distilbert-cos-v1')

Downloading:   0%|          | 0.00/737 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.5k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/333 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
sent1=converter.encode('manda um soba')

In [18]:
sent1.shape

(768,)

In [19]:
sentences = converter.encode(['bom dia pessoal','me veja um soba'])

In [20]:
for sent in sentences:
    print(np.linalg.norm(sent1-sent))

1.1086375
0.71818393


In [21]:
xtrain[:3]

['quero dois soba', 'quero um soba', 'quanto que foi?']

In [22]:
xval[:3]

['me veja o cardapio', 'quanto que deu?', 'quanto que foi?']

In [23]:
xtest[:5]

['vou querer 3 sobas',
 'vou querer 4 sobas',
 'vou querer 3 sobas',
 'vou querer 4 sobas',
 'vou querer 3 sobas']

In [24]:

xtrain_emb = converter.encode(xtrain)
xval_emb = converter.encode(xval)
xtest_emb = converter.encode(xtest)

In [25]:
xtrain[1],xtrain_emb[1][:10]

('quero um soba',
 array([-0.02123285,  0.02546068,  0.0097763 , -0.01808936,  0.0259546 ,
         0.01352183,  0.06349345,  0.04845588,  0.04439283,  0.0024159 ],
       dtype=float32))

# Indução do Modelo

In [26]:
import sklearn.neighbors as neighbors

In [27]:
model = neighbors.KNeighborsClassifier(n_neighbors=5)

In [28]:
model.fit(xtrain_emb,ytrain)

KNeighborsClassifier()

In [29]:
model.predict([converter.encode('gostaria de um soba')])

array(['fazer_pedido'], dtype='<U12')

# Ajuste de parâmetros

In [30]:
import sklearn.metrics as metrics

In [31]:
for i in range(1,15,2):
    model = neighbors.KNeighborsClassifier(n_neighbors=i,weights="distance")
    model.fit(xtrain_emb,ytrain)
    pred = model.predict(xval_emb)
    print(f"k={i}")
    print(metrics.classification_report(yval,pred))
    



k=1
              precision    recall  f1-score   support

fazer_pedido       1.00      1.00      1.00         7
fechar_conta       1.00      1.00      1.00         7
ver_cardapio       1.00      1.00      1.00         6

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

k=3
              precision    recall  f1-score   support

fazer_pedido       1.00      1.00      1.00         7
fechar_conta       1.00      1.00      1.00         7
ver_cardapio       1.00      1.00      1.00         6

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

k=5
              precision    recall  f1-score   support

fazer_pedido       1.00      1.00      1.00         7
fechar_conta       1.00      1.00      1.00         7
ver_cardapio       1.00      1.00      1.00         6

    accu

# Avaliação do Modelo

In [32]:
import sklearn.metrics as metrics

In [33]:
model = neighbors.KNeighborsClassifier(n_neighbors=9,weights="distance")
model.fit(xtrain_emb,ytrain)
pred = model.predict(xtest_emb)
print(f"k={i}")
print(metrics.classification_report(ytest,pred))

k=13
              precision    recall  f1-score   support

fazer_pedido       1.00      1.00      1.00        26
fechar_conta       1.00      1.00      1.00        26
ver_cardapio       1.00      1.00      1.00        26

    accuracy                           1.00        78
   macro avg       1.00      1.00      1.00        78
weighted avg       1.00      1.00      1.00        78



# Teste

In [34]:
model.predict([converter.encode('gostaria de pedir um soba')])

array(['fazer_pedido'], dtype='<U12')

#Modelo final

In [35]:
model.fit(converter.encode(xtrain_global+xtest),ytrain_global+ytest)

KNeighborsClassifier(n_neighbors=9, weights='distance')

In [36]:
from joblib import dump, load

In [37]:
dump(model, 'garcom.joblib') 

['garcom.joblib']

In [38]:
modelo_final = load('garcom.joblib') 

In [39]:
modelo_final.predict([converter.encode('me vega o menu')])

array(['ver_cardapio'], dtype='<U12')

# Construção de um chatbot

In [40]:
def convert_num(n):
    valores = {'um':1,
               'uma':1,
               'dois':2,
               'duas':2,
               'tres':3,
               'quatro':4,
               'cinco':5,
               'seis':6,
               'sete':7,
               'oito':8,
               'nove':9,
               'dez':10}
    ret = 0
    if n.isnumeric():
        ret = int(n)
    else:
        if n in valores.keys():
            ret = valores[n]

    return ret

In [41]:
!pip install unidecode

     |████████████████████████████████| 235 kB 6.3 MB/s 


In [42]:
from nltk.tokenize import TweetTokenizer
from unidecode import unidecode
tknzr = TweetTokenizer()

In [43]:
lista_entidades = [
'item:coca,cocas,coca-cola,coquinha,guarana,agua,guaranas',
'item:soba,espeto,sobas,espetos',
'num:1,2,3,4,5,6,7,8,9,10',
'num:um,dois,tres,quatro,cinco,seis,sete,oito,nove,dez,uma,duas'
]
entidades = dict()
def load_entidades(lista_entidades):
            for line in lista_entidades:
                entidade,valores = line.split(':')
                str_valores = valores[:]
                valores = str_valores.split(',')
                for valor in valores:
                    if valor not in entidades.keys():
                        entidades[valor] = entidade
load_entidades(lista_entidades)
def find_entidades(texto):
    ret = dict()
    for token in tknzr.tokenize(texto):
        token = token.lower()
        token = unidecode(token)
        if token in entidades.keys():
            ent = entidades[token]
            if ent not in ret.keys():
                ret[ent] = [token]
            else:
                ret[ent] += [token]
    return ret

In [44]:
def convert_num(n):
    valores = {'um':1,
               'uma':1,
               'dois':2,
               'duas':2,
               'tres':3,
               'quatro':4,
               'cinco':5,
               'seis':6,
               'sete':7,
               'oito':8,
               'nove':9,
               'dez':10}
    ret = 0
    if n.isnumeric():
        ret = int(n)
    else:
        if n in valores.keys():
            ret = valores[n]

    return ret

In [45]:
print(find_entidades('me ve um sobá ai'))

{'num': ['um'], 'item': ['soba']}


In [46]:
valor_cardapio = {
    'soba':20.0,
    'espeto':20.0,
    'coca':5.0,
    'guarana':5.0
}

In [47]:
def str_menu(h):
    rstr = ''
    for item in h:
        rstr += f"{item:<10}  {h[item]:>5}\n"
    return rstr


In [48]:
print(str_menu(valor_cardapio))

soba         20.0
espeto       20.0
coca          5.0
guarana       5.0



In [54]:
sair = False
#intencoes = ['fazer_pedido', 'fechar_conta', 'ver_cardapio']
print("Olá, sejá bem vindo ao Restaurante ___.\nMe chamo Jhonatan. O que vai querer hoje ?")
pedidos = []
while(not sair):
    cliente = input("")
    msg = ''
    pred = modelo_final.predict([converter.encode(cliente)])[0]
    if pred == 'ver_cardapio':
        msg = str_menu(valor_cardapio)
    elif pred == 'fazer_pedido':
        ent = find_entidades(cliente)
        print(ent)
        joined = [[x,y] for x,y in zip(ent['num'],ent['item'])]
        pedidos.append(joined)
        msg += 'ok '
        for n,item in joined[:-1]:
            msg += ',%s %s '%(n,item)
        msg += ('e %s %s '%(joined[-1][0],joined[-1][1]))        
        msg += 'saindo\n'
    elif pred == 'fechar_conta':
        msg += 'Ok, foi pedido:\n'
        total = 0
        for joined in pedidos:
            for n,item in joined:
                nvalor = convert_num(n)
                if item in valor_cardapio.keys():
                    vitem = valor_cardapio[item]
                total += vitem*nvalor
                msg += '%s %s %04.2f\n'%(n,item,vitem*nvalor)
            
        msg += ('Total foi R$ %4.2f\n'%total)
    elif pred == 'bye':
        msg = 'Bye, volte sempre\n'
        sair = True
    else:
        msg = 'Não, entendi. \n'+str_menu(valor_cardapio)
    print(msg)
        

Olá, sejá bem vindo ao Restaurante ___.
Me chamo Jhonatan. O que vai querer hoje ?
Olá Jhonatan, vou querer um espeto
{'num': ['um'], 'item': ['espeto']}
ok e um espeto saindo

muito obrigado, ficou quanto?
Ok, foi pedido:
um espeto 20.00
Total foi R$ 20.00

bye
soba         20.0
espeto       20.0
coca          5.0
guarana       5.0

tcah
{}


KeyError: ignored